In [ ]:
############################################## SETTINGS ##############################################  
import os
from os import walk
from tqdm import tqdm
import pickle
import random 
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
# Neuron
from   neuron           import h
import neuron as nrn
nrn.load_mechanisms('mechanisms/single')
# import custom functions
from   master_functions import *
#  Setup neuron
h.load_file('stdlib.hoc')
h.load_file('import3d.hoc')

############################################## SETTINGS ##############################################  
# original: original = True, method = 0 and physiological = False
# original = False        # inherited code
method = 0                # if 0 stimulate away from soma; if 1 then stimulate towards
physiological = True      # if False then uses original values else more physiological settings for phasic conductances 
# if model = 0 then original cell; if model = 1 then paper 
model = 1
cell_type='dspn'
specs = {'dspn': {
                    'N': 71,
                    'lib': 'Libraries/D1_71bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-dMSN_P270-20_1.02_SGA1-m24.swc',
                    },
         'ispn': {
                    'N': 34,
                    'lib': 'Libraries/D2_34bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-iMSN_P270-09_1.01_SGA2-m1.swc'}
        }


specs[cell_type]['model'] = model
morphology = specs[cell_type]['morph']
if model == 0:
    method = 0    
if model == 2:
    v_init = -84
else:
    v_init = -83.7155
return_currents = False
return_Ca = False
change_Ra = False
change_Cm = False
spine_per_length=1.711
change_spine_per_length = False

In [ ]:
sim = 13

In [ ]:
############################################### Figure 13 ###############################################   
# glutamate maximal
# vary glutamate timing
# Phasic glut
# model 1: 15 for subthreshold if gaba absent; 18 for standard
# model 1: 14 for subthreshold if gaba absent; 18? for standard
glut = True
if glut:
    if cell_type == 'dspn':
        num_gluts = 18
    elif cell_type == 'ispn':
        num_gluts = 15
else:
    num_gluts = 0

g_AMPA = 0.001 # original setting 0.001
AMPA = True
NMDA = True
ratio = 2.15 # NMDA to AMPA conductance ratio (not necessarily the current ratio)
glut_placement = 'distal'
if physiological:
    gaba_weight = 0.000005
else:
    gaba_weight = 0.0025 # tuned for non-distributed gaba (will be higher depolarization than ~5mv if running whole cell)

gaba_reversal = -60 # -83.7155 # 
# Phasic gaba
gaba = False
num_gabas = 15
if not gaba:
    num_gabas = 0
    gaba_weight = 0
protocol = 'upstate vs distance with GABA_{}'.format(gaba_weight)
####
sim_time = 400
vary_gaba = False # if true timing of gaba is varied relative to glut; if false then vary glut relative to gaba    
# if vary_gaba is True then gaba timing is varied and glut timing is given by stim_time
# if vary_gaba is False then glut timing is varied and gaba timing is given by stim_time
stim_time = 150

spine_neck_diam = 0.1 # original setting 0.1

impedance = False
# impedance is defined in frequency domain
freq = 10 

save = False

In [ ]:
# Save figures for dist sim
def save_all2():
    import datetime 
    time = datetime.datetime.now()
    if os.name == 'nt':
        time = time.strftime("%Y-%m-%d_%H-%M-%S")

    path_cell = "{}".format(cell_type)
    if not os.path.exists(path_cell):
        os.mkdir(path_cell)    
    path1 = "{}/model{}".format(path_cell, model)
    if not os.path.exists(path1):
        os.mkdir(path1)
        
    if not os.path.exists(path1):
        os.mkdir(path1)
    if physiological: 
        path2 = "{}/physiological".format(path1)
    else:
        path2 = "{}/nonphysiological".format(path1)
    if not os.path.exists(path2):
        os.mkdir(path2)  
    path3 = "{}/images".format(path2)
    if not os.path.exists(path3):
        os.mkdir(path3)
    path4 = "{}/simulations".format(path2)
    if not os.path.exists(path4):
        os.mkdir(path4)  
    path5 = "{}/sim{}".format(path3, sim)
    if not os.path.exists(path5):
        os.mkdir(path5)    
    path6 = "{}/sim{}".format(path4, sim)
    if not os.path.exists(path6):
        os.mkdir(path6)  

    image_dir = "{}/{}".format(path5, protocol)
    if not os.path.exists(image_dir):
        os.mkdir(image_dir)  
    sim_dir = "{}/{}".format(path6, protocol)
    if not os.path.exists(sim_dir):
        os.mkdir(sim_dir)  

    with open('{}/{}_spines.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(num_spines, handle, protocol=pickle.HIGHEST_PROTOCOL) 
    with open('{}/{}_distances.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(distances, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_midpoints.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(midpoints, handle, protocol=pickle.HIGHEST_PROTOCOL)    

    with open('{}/{}_upstates.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(upstate_duration, handle, protocol=pickle.HIGHEST_PROTOCOL) 
    with open('{}/{}_v_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_v_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_vdend_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_vdend_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_v_tree.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_v_tree, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_Ca_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_Ca_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_Cadend_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_Cadend_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_Ca_tree.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_Ca_tree, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_name_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_name_data, handle, protocol=pickle.HIGHEST_PROTOCOL)        
    with open('{}/{}_t_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_t_data, handle, protocol=pickle.HIGHEST_PROTOCOL)        
    if impedance:
        with open('{}/{}_imp_dend_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
            pickle.dump(pickle_imp_dend_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('{}/{}_imp_soma_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
            pickle.dump(pickle_imp_soma_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

    plt.scatter(midpoints, upstate_duration)
    ax = plt.gca()
    ax.set_ylim([0, 120])
    plt.savefig('{}/{}_distances_vs_upstate_duration.pdf'.format(image_dir, protocol))
    plt.show()


In [ ]:
def distance_plots(protocol, sim_time, num_gluts, glut_onsets, gaba, gaba_weight, plot=False, path=True, cell_type='dspn', show=True):
    # Upstate lists
    distances = []
    midpoints = []
    upstate_duration = []
    num_spines = []
    pickle_name_data = []
    pickle_v_data = []
    pickle_vdend_data = []         
    pickle_v_tree = {}
    pickle_t_data = []  
    pickle_imp_dend_data = []
    pickle_imp_soma_data = []
    pickle_Ca_data = []
    pickle_Cadend_data = []         
    pickle_Ca_tree = {}

    # Create cell
    if sim in [17,22]:
        cell, spines, dend_tree, branch_groups = cell_build(cell_type=cell_type, specs=specs, addSpines=True, branch=True, spine_per_length=spine_per_length)
    else:
        cell, spines, dend_tree, branch_groups = cell_build(cell_type=cell_type, specs=specs, addSpines=True, branch=True)
    
    if change_Ra:
        space_clamped(cell=cell, spines=spines, Ra=Ra)
        print(f"Ra is {Ra} ΜΩ")
    if change_Cm:
        cap(cell=cell, spines=spines, cm = Cm)
        print(f"Cm is {Cm} μFcm\u207B\u00B2")

    # Make list of dendrite sections with at least 3 spines
    dends_with_spines = dend_spine_selector(cell=cell, spines=spines, branch_groups=branch_groups)

#     for shortening protocol for testing
#     dends_with_spines = []
#     for dend in cell.dendlist:
#         if dend.name() in ['dend[15]', 'dend[25]', 'dend[45]']:
#             dends_with_spines.append(dend)


    # Iterate through each dendrite with spines
    for dend in tqdm(dends_with_spines):
        # Distance from soma
        distance = round(h.distance(dend(0.5)), 2)

        # Number of spines in section
        num_spines.append(len(list(spines[dend.name()].items())))

        # Add dendrite section 
        glut_secs = []
        glut_secs.append(dend)
        glut_secs *= num_gluts # need to duplicate sections to place synapses

        # Place glutamatergic synapses
        glut_synapses, glut_stimulator, glut_connection, ampa_currents, nmda_currents = glut_place(
                            spines=spines,
                            method=method, 
                            physiological=physiological, 
                            AMPA=AMPA, 
                            g_AMPA=g_AMPA,
                            NMDA=NMDA,
                            ratio=ratio,
                            glut_time=glut_time,
                            glut_secs=glut_secs,
                            glut_onsets=glut_onsets,
                            num_gluts=num_gluts,
                            return_currents=return_currents)

        
        synapse_dists = synapse_dist(spines=spines, method=method, glut_secs=glut_secs, num_gluts=num_gluts)
        midpoint = sum(synapse_dists) / len(synapse_dists)
        midpoints.append(midpoint)
        
        # add TONIC gaba
        if gaba:
            num_gabas = sim_time
            gaba_time = 0
            gaba_onsets = list(range(gaba_time, gaba_time + int(num_gabas)+1)) * 58 # factor accounting for each dendritic branch
        else:
            num_gabas = []
            gaba_time = None
            gaba_onsets = None
            
        gaba_secs = gaba_add(cell=cell, gaba=gaba, gaba_placement='everywhere', num_gabas=num_gabas, show=show)
        # Place gabaergic synapses
        gaba_synapses, gaba_stimulator, gaba_connection, gaba_currents, gaba_conductances = gaba_place(
               physiological = physiological,
               gaba_reversal = gaba_reversal,
               gaba_weight = gaba_weight,
               gaba_time = gaba_time,
               gaba_secs = gaba_secs,
               gaba_onsets = gaba_onsets,
               num_gabas = num_gabas,
               return_currents = return_currents,
               show=show)

        # Record vectors
        if path:
            pathlist = path_finder(cell=cell, dend_tree=dend_tree, dend=dend)
            v = record_v(cell=cell, seclist=pathlist, loc=0.4)
            ca = record_cai(cell, seclist=pathlist, loc=0.4, return_Ca=return_Ca)
        else:
            v = record_all_v(cell, loc=0.4)
            ca = record_all_cai(cell, seclist=pathlist, loc=0.4, return_Ca=return_Ca)
        t = h.Vector().record(h._ref_t)  
        i_mechs_dend = record_i_mechs(cell=cell, dend=dend, loc=0.4, return_currents=return_currents)
        # Initialize cell starting voltage
        h.finitialize(v_init)
        # Run simulation
        dt = 0.025
        h.dt = dt
        if impedance:
            # set up vectors
            imp, zvec1, zvec2 = record_impedance(dend=dend, loc=0.4)
            freq = 10
            while h.t < sim_time:                
                imp.compute(freq,1)            
                zvec1.append(imp.input(0.4, sec=dend))    
                zvec2.append(imp.input(0.4, sec=cell.soma))    
                h.fadvance()
        else:
            while h.t < sim_time:
                h.fadvance()
        # store values
        pickle_v_data.append(np.array(v['soma[0]']))
        pickle_vdend_data.append(np.array(v[dend.name()]))
        pickle_v_tree['{}'.format(dend.name())] = pd.DataFrame(v)
        pickle_t_data.append(np.array(t))
        pickle_name_data.append(dend.name())
        if return_Ca:
            pickle_Ca_data.append(np.array(ca['soma[0]']))
            pickle_Cadend_data.append(np.array(ca[dend.name()]))
            pickle_Ca_tree['{}'.format(dend.name())] = pd.DataFrame(ca)
        
        if impedance:
            pickle_imp_dend_data.append(np.asarray(zvec1))
            pickle_imp_soma_data.append(np.asarray(zvec2))

        # Calculate quarterdrop upstate duration (adapted implementation from Shenyu)
        # Baseline voltage calculation
        up_duration = []
        bs1_start = glut_time - 20
        bs1_end = glut_time # start of stimulation

        # Convert to numpy
        v_data = np.asarray(v['soma[0]'])
        t_data = np.asarray(t)

        # Mean voltage for baseline
        bsl_V = v_data[(t_data >= bs1_start) & (t_data <= bs1_end)].mean() 
        # Peak voltage
        peak_V = v_data[(t_data >= bs1_start) & (t_data <= sim_time)].max() - bsl_V
        # Quarter drop voltage value
        quarterdrop = 0.75*(peak_V)+bsl_V
        # Normalized v_data
        dtoQuarterDrop = abs(v_data - quarterdrop)
        # Peak index
        peak_idx = np.argmax(v_data)
        # Closest time point to calculated quarterdrop
        X_closest = t_data[peak_idx + np.argmin(dtoQuarterDrop[peak_idx:])]
        # Total quarterdrop time
        up_duration = X_closest - glut_time -  num_gluts # from end of stim to quarter drop
        upstate_duration.append(up_duration)
        distances.append(distance)
        
        plot_dend = glut_secs[0] # first glut section is returned as v_dend
        if plot:
            # gets secs to plot; if path is True, plots PSPs in each dendrite that forms part of dendritic path to soma
            if path:
                seclist = pathlist
            else:
                seclist = dend2plot(cell)
            fig_path = plot1(cell=cell, dend=dend, t=t, v=v, seclist=seclist, sparse=False, protocol='')
            fig_path.show()

    return distances, midpoints, upstate_duration, num_spines, pickle_v_data, pickle_vdend_data, pickle_v_tree, pickle_Ca_data, pickle_Cadend_data, pickle_Ca_tree, pickle_imp_soma_data, pickle_imp_dend_data, pickle_name_data, pickle_t_data


In [ ]:
plot = False
path = True
if sim==14:
    show = False
else:
    show = True
glut_time = stim_time
glut_onsets = list(range(glut_time, glut_time + num_gluts)) 
distances, midpoints, upstate_duration, num_spines, pickle_v_data, pickle_vdend_data, pickle_v_tree, pickle_Ca_data, pickle_Cadend_data, pickle_Ca_tree, pickle_imp_soma_data, pickle_imp_dend_data, pickle_name_data, pickle_t_data = distance_plots(protocol=protocol, sim_time=sim_time, num_gluts=num_gluts, glut_onsets=glut_onsets, gaba=gaba, gaba_weight=gaba_weight, plot=plot, path=path, cell_type=cell_type, show=show)
save_all2()